# Robust Optimal Experiments

The Randomized Controlled Trial (RCT) is a trusted method in experimental design that aims to figure out responses to certain interventions, while reducing the discrepancy in results due to variance in subjects. In fact, in 2019, Prof. Duflo and Banerjee from MIT got the Nobel Prize in Economics for addressing questions/issues in development economics (esp. poverty and availability of healthcare) using RCTs. 

But very few people talked about the fact that RCTs are quite ineffective in one major aspect: They rely on the Central Limit Theorem to ensure that control and experiments have similar distributions of traits. This requires large experimental populations, which are expensive and often unavailable. 

Instead, there is research to suggest that **optimal experimental design (OED)** can be significantly more powerful. 

This demo will hopefully demonstrate that randomization is NOT a reliable method for getting the right distribution of "features" in subjects. Furthermore, it will demonstrate the influence of robustness on optimal experimental design. 

## Motivational Problem: Medical Trials

Suppose that we only have the budget to conduct initial Covid-19 vaccine trials on 10 patients, where the patients are split 50/50 between control and treatment groups. We have had 20 applicants with 5 traits, generated randomly in this instance. (We have chosen small numbers since this problem can quickly become computationally challenging. But it is definitely solvable in larger scale as well.) 

First, we initialize our computational environment.

In [ ]:
# Activating Julia environment
using Pkg
Pkg.activate(".")
using JuMP, Distributions, Random, LinearAlgebra, Gurobi, Plots

Then, we generate some random patients, with traits being sampled from the standard normal. 

In [ ]:
function generate_random_people(n_people::Int64 = 20, n_traits::Int64 = 5)
    # NOTE THAT OUR DATA IS NORMALIZED, so it makes the formulation more straight-forward. 
    continuous_values = rand(MersenneTwister(314), Normal(0.00, 1), (n_people, n_traits))
    return continuous_values
end
n_groups = 2
n_patients = 10
n_people = 20
n_traits = 5
data = generate_random_people(20, 5)

Initially, we will create our control and experiment groups by simply picking the first 5 to be in the control, and the next 5 to be in the experiment groups. Then, we can evaluate the means and variances of the traits of patients in each group, and compare then. 
(For simplicity, we will only consider the diagonal of the covariance matrix. However, this method can be extended to the full covariance matrix, by adding a lot more variables!)

In [ ]:
# Experimental design through randomization
ctrl_idxs = Int64.(collect(1:n_patients/2))
vacc_idxs = Int64.(collect(n_patients/2+1:n_patients))
function print_details(data, ctrl_idxs, vacc_idxs)
    println("Control group: ", ctrl_idxs)
    println("Vaccine group: ", vacc_idxs)
    println("Mean traits of control group: ", round.(mean(data[ctrl_idxs, :], dims=1); sigdigits = 4))
    println("Mean traits of vaccine group: ", round.(mean(data[vacc_idxs, :], dims=1); sigdigits = 4))
    println("Var of traits of control group: ", round.(var(data[ctrl_idxs, :], dims=1); sigdigits = 4))
    println("Var of traits of vaccine group: ", round.(var(data[vacc_idxs, :], dims=1); sigdigits = 4))
    println("Nominal objective: ", round.(sum(abs.(mean(data[ctrl_idxs, :], dims=1) - mean(data[vacc_idxs, :], dims=1))) + 
            0.5 * sum(abs.(var(data[ctrl_idxs, :], dims=1) - var(data[vacc_idxs, :], dims=1))); sigdigits = 4))
    return
end
print_details(data, ctrl_idxs, vacc_idxs)

Our objective function in this case is to minimize the absolute difference of the sum of means, and 1/2 times the absolute different of the sum of variances of traits between the control and experiment group. Note that we are not limited to this objective function; feel free to experiment in your own time with others. (For example, we could try minimizing variance while keeping the mean variation below a threshold... you can try any combination that is bounded!)

We can examine the errors more specifically by plotting them.

In [ ]:
# Plotting the errors
function plot_errors(data, ctrl_idxs, vacc_idxs)
    l = @layout [a ; b]
    p1 = bar(collect(1:n_traits), mean(data[ctrl_idxs, :], dims=1)' .-  mean(data[vacc_idxs, :], dims=1)', label = "Mean errors")
    p2 = bar(collect(1:n_traits), var(data[ctrl_idxs, :], dims=1)' .- var(data[vacc_idxs, :], dims=1)', label = "Variance errors")
    plt = plot(p1, p2, layout = l)
    return plt
end
plt = plot_errors(data, ctrl_idxs, vacc_idxs)

Clearly, there are some discrepancies between the means and variances of the groups.

### Can Optimization do better? 
It sure can! Let's start by writing out the problem in a mixed-integer optimization format. 

In this case, we will pick 2 groups of equal numbers of patients from the population, while minimizing the weighted L1-norm error in the mean and variances between the two groups. 

In [ ]:
# Let's start creating out model, and trying to solve without uncertainty
m = Model(Gurobi.Optimizer)
set_optimizer_attribute(m, "OutputFlag", 0)
@variable(m, x[i=1:n_people, 1:n_groups], Bin)
@variable(m, μ_p[i=1:n_groups, j=1:n_traits]) # Mean
@variable(m, σ_p[i=1:n_groups, j=1:n_traits]) # Variance
for j = 1:n_groups # Taking the mean and std deviation of parameters for each group
    @constraint(m, μ_p[j,:] .== 1/(n_patients/n_groups) * 
                    sum(data[i,:] .* x[i,j] for i=1:n_people))
    @constraint(m, σ_p[j,:] .== 1/(n_patients/n_groups) * 
                    sum(data[i,:].^2 .* x[i,j] for i=1:n_people))
    @constraint(m, sum(x[:,j]) == n_patients/n_groups)
end
for i = 1:n_people
    @constraint(m, sum(x[i, :]) <= 1) # each patient only picked at most once
end

@variable(m, d)
@variable(m, M[1:n_traits]) # mean error
@variable(m, V[1:n_traits]) # variance error
rho = 0.5
@objective(m, Min, sum(M) + rho*sum(V))
for i = 1:n_groups
    for j = i+1:n_groups
        @constraint(m, M[:] .>= μ_p[i,:] - μ_p[j,:])
        @constraint(m, M[:] .>= μ_p[j,:] - μ_p[i,:])
        @constraint(m, V[:] .>= σ_p[i, :] - σ_p[j, :])
        @constraint(m, V[:] .>= σ_p[j, :] - σ_p[i, :])
    end
end
optimize!(m)

Let's see the results, and how our patients (by index) have changed compared to randomization.

In [ ]:
# Results
ctrl_opt = findall(x -> x == 1, Array(value.(x[:,1])))
vacc_opt = findall(x -> x == 1, Array(value.(x[:,2])))
print_details(data, ctrl_opt, vacc_opt)

We see a more than 10x improvement in the objective function! We can also see the reduction in mean and variance errors graphically. 

In [ ]:
# Plotting the distribution
plot_errors(data, ctrl_opt, vacc_opt)

### How does Robust Optimization (RO) change our solutions? 
It is very possible that there is some error in the values of the traits of each subject. These errors could arise from a variety of sources: measurement error, observer's paradox, or even due to subjects lying in order to influence their perceived likelihood of being in the treatment group!

We will deal with this kind of uncertainty through **robust OED**. Robust OED will assume that there is uncertainty in the data in the problem that is defined by an uncertainty set. One potential description of the uncertainty is a **budget uncertainty set**, where the maximum absolute deviation of each trait for each subject is constrained ($||\mathbf{z}||_{\infty} \leq \rho$), as well as the cumulative absolute deviations of each trait for all subjects ($||\mathbf{z}||_{1} \leq \Gamma$). In this case, we assume such an uncertainty set, with an maximum absolute deviation $\rho = 1$, and a cumulative deviation $\Gamma = 2$. Intuitively, this would allow up to 20 trait perturbations of magnitude 0.1, or 40 perturbations of magnitude 0.05, and so on. 

We start our optimization model similarly, by defining the means and variances, as well as the objective function. 

In [ ]:
# Let's start creating out model, and trying to solve with uncertainty
rm = Model(Gurobi.Optimizer)
set_optimizer_attribute(rm, "OutputFlag", 0)
@variable(rm, x[i=1:n_people, 1:n_groups], Bin)
@variable(rm, μ_p[i=1:n_groups, j=1:n_traits]) # Mean
@variable(rm, σ_p[i=1:n_groups, j=1:n_traits]) # Variance
ρ = 0.1
Γ = 2
for j = 1:n_groups # Taking the mean and std deviation of parameters for each group
    @constraint(rm, μ_p[j,:] .== 1/(n_patients/n_groups) * 
                    sum(data[i,:].*x[i,j] for i=1:n_people))
    @constraint(rm, σ_p[j,:] .== 1/(n_patients/n_groups) * 
                    sum(data[i,:].^2 .* x[i,j] for i=1:n_people))
    @constraint(rm, sum(x[:,j]) == n_patients/n_groups)
end
for i = 1:n_people
    @constraint(rm, sum(x[i, :]) <= 1)
end
@variable(rm, M[1:n_traits])
@variable(rm, V[1:n_traits])
@objective(rm, Min, sum(M) + 0.5*sum(V))


Note that we have defined some variables $\mathbf{M}$ and $\mathbf{V}$ to describe the errors. We cannot directly embed the uncertainty into means $\mu$ and variances $\sigma$, because these are defined by linear inequalities. Adding robustness to equalities would make this problem infeasible. Instead, we will minimize the **worst-case** errors as described by $\mathbf{M}$ and $\mathbf{V}$, by embedding the budget uncertainty set into the model through its robust counterpart. 

In [ ]:
# With the following budget uncertainty
# @uncertain(rm, ell[1:n_people, 1:n_traits])
# @constraint(rm, norm(ell, 1) <= Γ)
# @constraint(rm, -ρ .<= ell .<= ρ)  
# Let's use the robust counterpart
for i = 1:n_groups # We embed the uncertainty in the errors!
    for j = i+1:n_groups
        for l = 1:n_traits
            y = @variable(rm, [1:n_traits])
            normdummy = @variable(rm, [1:n_traits])
            @constraint(rm, normdummy .>= y)
            @constraint(rm, normdummy .>= -y)
            infdummy = @variable(rm)
            @constraint(rm, [k = 1:n_traits], infdummy >= (x[k,j] - x[k,i] - y[k]))
            @constraint(rm, [k = 1:n_traits], infdummy >= -(x[k,j] - x[k,i] - y[k]))
            @constraint(rm, M[l] * n_patients/n_groups >= 
                        sum(data[k,l] .* (x[k,j] - x[k,i]) for k=1:n_people) + 
                        ρ * sum(normdummy) + Γ*infdummy)
            y = @variable(rm, [1:n_traits])
            normdummy = @variable(rm, [1:n_traits])
            @constraint(rm, normdummy .>= y)
            @constraint(rm, normdummy .>= -y)
            infdummy = @variable(rm)
            @constraint(rm, [k = 1:n_traits], infdummy >= (x[k,j] - x[k,i] - y[k]))
            @constraint(rm, [k = 1:n_traits], infdummy >= -(x[k,j] - x[k,i] - y[k]))
            @constraint(rm, M[l] * n_patients/n_groups >= 
                        - sum(data[k,l] .* (x[k,j] - x[k,i]) for k=1:n_people) +  
                        ρ * sum(normdummy) + Γ*infdummy)
#             Sometimes you have to get creative... linearization of the change of the variance. 
            y = @variable(rm, [1:n_traits])
            normdummy = @variable(rm, [1:n_traits])
            @constraint(rm, normdummy .>= y)
            @constraint(rm, normdummy .>= -y)
            infdummy = @variable(rm)
            @constraint(rm, [k = 1:n_traits], infdummy >= (2*data[k,l]*(x[k,j] - x[k,i]) - y[k]))
            @constraint(rm, [k = 1:n_traits], infdummy >= -(2*data[k,l]*(x[k,j] - x[k,i]) - y[k]))
            @constraint(rm, V[l] * n_patients/n_groups >= 
                        sum(data[k,l].^2 .* (x[k,j] - x[k,i]) for k=1:n_people) + 
                        ρ*sum(normdummy) + Γ*infdummy)
            y = @variable(rm, [1:n_traits])
            normdummy = @variable(rm, [1:n_traits])
            @constraint(rm, normdummy .>= y)
            @constraint(rm, normdummy .>= -y)
            infdummy = @variable(rm)
            @constraint(rm, [k = 1:n_traits], infdummy >= (2*data[k,l]*(x[k,j] - x[k,i]) - y[k]))
            @constraint(rm, [k = 1:n_traits], infdummy >= -(2*data[k,l]*(x[k,j] - x[k,i]) - y[k]))
            @constraint(rm, V[l] * n_patients/n_groups >= 
                        -sum(data[k,l].^2 .* (x[k,j] - x[k,i]) for k=1:n_people) + 
                        ρ*sum(normdummy) + Γ*infdummy)
        end
    end
end

In [ ]:
optimize!(rm)

It seems like we have found a design of experiments that is robust to the budget uncertainty set! Let's take a look at the results. 

In [ ]:
ctrl_ro = findall(x -> x == 1, Array(value.(x[:,1])))
vacc_ro = findall(x -> x == 1, Array(value.(x[:,2])))
print_details(data, ctrl_ro, vacc_ro)
println("Robust objective: ", objective_value(rm))

We observe slight changes in the optimal allocations of patients. In addition, the nominal objective is worsened by around 80\% under robustness considerations. However, in presence of uncertainty, this new experimental design makes sure that the worst case outcome of our objective function is only around 25\% worse than the nominal case. The OED without robustness is a lot more sensitive to this uncertainty. Below, we compute the worst-case objective of the random, optimal and robust cases for comparison. 

In [ ]:
# Nominal worst case
constrs = [@constraint(rm, x[ctrl_idxs,1] .== ones(5))..., @constraint(rm, x[vacc_idxs, 2] .== ones(5))...]
optimize!(rm)
nom_worst = objective_value(rm);
println("Random worst-case: " * string(nom_worst))
delete.(rm, constrs);
constrs = [@constraint(rm, x[ctrl_opt,1] .== ones(5))..., @constraint(rm, x[vacc_opt, 2] .== ones(5))...]
optimize!(rm)
opt_worst = objective_value(rm)
println("Optimal worst-case: " * string(opt_worst))
delete.(rm, constrs);
constrs = [@constraint(rm, x[ctrl_ro,1] .== ones(5))..., @constraint(rm, x[vacc_ro, 2] .== ones(5))...]
optimize!(rm)
ro_worst = objective_value(rm)
println("Robust optimal worst-case: " * string(ro_worst))


What we see is that the robust optimal solution definitively has the lowest worst-case objective, and while the optimal solution (without uncertainty) has a better nominal outcome, it is a lot more sensitive to uncertainties in the traits of the subjects. The randomized cases is again by far the worst, having the worst performance both without and with uncertainty. 

For curiosity's sake, we also plot the mean and variance errors for the robust solution. 

In [ ]:
plot_errors(data, ctrl_ro, vacc_ro)

# Conclusions

- Optimal experimental design is a useful method to make sure that the moments of our experiment and control groups are similar, while still being representative of the global population. 
- Uncertainty can result from a variety of factors in experimental designs. 
- Robust optimal experimental design can improve the efficacy of experiments with small reduction in nominal performance compared to optimized groups without uncertainty. However, robust OED gives significantly better worst-case outcomes, and reduces the sensitivity of experiments to uncertainty in the traits of control and experiment subjects!
- Randomization is consistently the **worst** method for experimental design by far!